### Load Data

In [1]:
# TODO below lines are not for production
import warnings
warnings.filterwarnings('ignore')

In [2]:
PROJECT_PATH = "/Users/vinay/Projects/my-mlops/"

In [7]:
import pandas as pd
from datasets import Dataset
import os

df = pd.read_csv(os.path.join(PROJECT_PATH, "data", "IMDB-Dataset.csv"))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.3)
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 15000
    })
})

In [9]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [12]:
label2id = {'negative': 0, 'positive': 1}
id2label = {0: 'negative', 1: 'positive'}

dataset = dataset.map(lambda x: {'label': label2id[x['sentiment']]})

Map: 100%|██████████| 15000/15000 [00:00<00:00, 60332.57 examples/s]


In [14]:
dataset['train'][0]

{'review': "This movie actually hurts to watch. Not only did I not laugh once, I ended up getting a serious headache. At times, I felt so sorry for the actors involved. The best way to sum it up is to note that among sex comedies, it is probably the least funny and least sexy of all time. I'm only sorry I can't give it a vote of less than one. To give this a rating of one is an insult to every movie that scores two or better.<br /><br />Now I see that they're filming a sequel. Hearing that someone is actually paying these people to make another movie convinces me that there's just too much money in Hollywood. RAISE THEIR TAXES!!! Making a sequel to this movie may very well constitute a crime against humanity; perhaps an international tribunal should be convened, or U.N. sanctions applied to the filmmakers.<br /><br />In short, it's a really bad movie. Really, really.",
 'sentiment': 'negative',
 'label': 0}

### Data Tokenization

In [19]:
from transformers import AutoTokenizer
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_ckpt = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

In [ ]:
def tokenize(batch):
    temp = tokenizer(batch['review'], padding=True, Truncation=True, max_length=300)
    return temp

dataset = dataset.map(tokenize, batched=True, batch_size=None)

{'input_ids': [101, 2023, 3185, 2941, 13403, 2000, 3422, 1012, 2025, 2069, 2106, 1045, 2025, 4756, 2320, 1010, 1045, 3092, 2039, 2893, 1037, 3809, 14978, 1012, 2012, 2335, 1010, 1045, 2371, 2061, 3374, 2005, 1996, 5889, 2920, 1012, 1996, 2190, 2126, 2000, 7680, 2009, 2039, 2003, 2000, 3602, 2008, 2426, 3348, 22092, 1010, 2009, 2003, 2763, 1996, 2560, 6057, 1998, 2560, 7916, 1997, 2035, 2051, 1012, 1045, 1005, 1049, 2069, 3374, 1045, 2064, 1005, 1056, 2507, 2009, 1037, 3789, 1997, 2625, 2084, 2028, 1012, 2000, 2507, 2023, 1037, 5790, 1997, 2028, 2003, 2019, 15301, 2000, 2296, 3185, 2008, 7644, 2048, 2030, 2488, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2085, 1045, 2156, 2008, 2027, 1005, 2128, 7467, 1037, 8297, 1012, 4994, 2008, 2619, 2003, 2941, 7079, 2122, 2111, 2000, 2191, 2178, 3185, 19480, 2033, 2008, 2045, 1005, 1055, 2074, 2205, 2172, 2769, 1999, 5365, 1012, 5333, 2037, 7773, 999, 999, 999, 2437, 1037, 8297, 2000, 2023, 3185, 2089, 2200, 2092, 12346, 1037, 4126, 2114,

In [ ]:
dataset['train'][0].keys()